In [ ]:
!pip install opencv-python pydicom numpy scikit-learn pandas tensorflow flwr
!pip install efficientnet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 598.1/598.1 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstalled click-8.2.1
  Attempting uninstall: cryptography
    Found existing ins

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 4.6 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import cv2

def resize_npy_images(input_folder, output_folder, size=(224, 224)):
    os.makedirs(output_folder, exist_ok=True)
    for filename in os.listdir(input_folder):
        if filename.endswith(".npy"):
            img = np.load(os.path.join(input_folder, filename))
            resized_img = cv2.resize(img, size)
            np.save(os.path.join(output_folder, filename), resized_img)


In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models

def create_model(input_shape=(224, 224, 3)):
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split

input_dir = "/content/drive/MyDrive/LIDC-NPY"
output_dir = "/content/drive/MyDrive/LungCancerFLData1"
os.makedirs(output_dir, exist_ok=True)

def load_and_preprocess_images():
    images = []
    labels = []  # Replace with actual labels if available
    for file in os.listdir(input_dir):
        if file.endswith(".npy"):
            path = os.path.join(input_dir, file)
            try:
                img = np.load(path)

                # Handle both 2D and 3D
                if len(img.shape) == 3:
                    # Take middle slice if it's 3D (z, x, y)
                    mid_slice = img[img.shape[0] // 2]
                elif len(img.shape) == 2:
                    mid_slice = img
                else:
                    print(f"Skipping {file} due to unexpected shape: {img.shape}")
                    continue

                # Resize to 224x224 and convert to 3 channels
                img_resized = cv2.resize(mid_slice, (224, 224))
                img_resized = np.stack([img_resized]*3, axis=-1)

                images.append(img_resized)
                labels.append(1 if "malign" in file.lower() else 0)  # Dummy logic

            except Exception as e:
                print(f"⚠️ Skipping {file} due to error: {e}")

    return np.array(images), np.array(labels)

def split_and_save(images, labels, num_clients=4):
    # Shuffle the entire dataset once
    indices = np.arange(len(images))
    np.random.shuffle(indices)
    images, labels = images[indices], labels[indices]

    # Split into train+test
    x_train_all, x_test_all, y_train_all, y_test_all = train_test_split(
        images, labels, test_size=0.2, stratify=labels
    )

    # Split train into non-iid partitions for each client
    train_size_per_client = len(x_train_all) // num_clients
    test_size_per_client = len(x_test_all) // num_clients

    for i in range(num_clients):
        train_start = i * train_size_per_client
        train_end = (i + 1) * train_size_per_client
        test_start = i * test_size_per_client
        test_end = (i + 1) * test_size_per_client

        x_train = x_train_all[train_start:train_end]
        y_train = y_train_all[train_start:train_end]
        x_test = x_test_all[test_start:test_end]
        y_test = y_test_all[test_start:test_end]

        client_dir = os.path.join(output_dir, f"client{i+1}")
        os.makedirs(client_dir, exist_ok=True)

        np.save(os.path.join(client_dir, "x_train.npy"), x_train)
        np.save(os.path.join(client_dir, "y_train.npy"), y_train)
        np.save(os.path.join(client_dir, "x_test.npy"), x_test)
        np.save(os.path.join(client_dir, "y_test.npy"), y_test)

        print(f"\n📦 Client{i+1} - Training Samples: {len(x_train)}")
        print(f"🧪 Client{i+1} - Testing Samples: {len(x_test)}")

if __name__ == "__main__":
    images, labels = load_and_preprocess_images()
    split_and_save(images, labels)



✅ Saved client datasets in: /content/drive/MyDrive/LungCancerFLData


✅ Saved client datasets in: /content/drive/MyDrive/LungCancerFLData



In [ ]:
import os
import numpy as np

client_root = "/content/drive/MyDrive/LungCancerFLData1"
num_clients = 4

for i in range(1, num_clients + 1):
    client_dir = os.path.join(client_root, f"client{i}")
    x_train_path = os.path.join(client_dir, "x_train.npy")
    x_test_path = os.path.join(client_dir, "x_test.npy")

    if os.path.exists(x_train_path):
        x_train = np.load(x_train_path)
        print(f"📦 Client{i} - Training Samples: {len(x_train)}")
    if os.path.exists(x_test_path):
        x_test = np.load(x_test_path)
        print(f"🧪 Client{i} - Testing Samples: {len(x_test)}\n")



📦 Client1 - Training Samples: 164
🧪 Client1 - Testing Samples: 41

📦 Client2 - Training Samples: 164
🧪 Client2 - Testing Samples: 41

📦 Client3 - Training Samples: 164
🧪 Client3 - Testing Samples: 41

📦 Client4 - Training Samples: 164
🧪 Client4 - Testing Samples: 41



In [ ]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split

# Set paths
input_dir = "/content/drive/MyDrive/LIDC-NPY"
output_dir = "/content/drive/MyDrive/LungCancerFLData1"
os.makedirs(output_dir, exist_ok=True)

# Function to load and preprocess DICOM slices
def load_and_preprocess_images():
    images = []
    labels = []
    for file in os.listdir(input_dir):
        if file.endswith(".npy"):
            path = os.path.join(input_dir, file)
            try:
                img = np.load(path)

                # Take middle slice if 3D
                if len(img.shape) == 3:
                    mid_slice = img[img.shape[0] // 2]
                elif len(img.shape) == 2:
                    mid_slice = img
                else:
                    print(f"⚠️ Skipping {file}: Unexpected shape {img.shape}")
                    continue

                # Resize to 224x224 and stack to 3 channels (RGB)
                img_resized = cv2.resize(mid_slice, (224, 224))
                img_rgb = np.stack([img_resized]*3, axis=-1)

                images.append(img_rgb)
                labels.append(1 if "malign" in file.lower() else 0)  # Dummy label logic

            except Exception as e:
                print(f"⚠️ Error loading {file}: {e}")

    return np.array(images), np.array(labels)

# Function to split data and save to client folders
def split_and_save(images, labels, num_clients=4):
    # Shuffle dataset
    indices = np.arange(len(images))
    np.random.shuffle(indices)
    images, labels = images[indices], labels[indices]

    # Global train/test split
    x_train_all, x_test_all, y_train_all, y_test_all = train_test_split(
        images, labels, test_size=0.2, stratify=labels
    )

    # Split train and test sets for clients
    train_size_per_client = len(x_train_all) // num_clients
    test_size_per_client = len(x_test_all) // num_clients

    for i in range(num_clients):
        train_start = i * train_size_per_client
        train_end = (i + 1) * train_size_per_client
        test_start = i * test_size_per_client
        test_end = (i + 1) * test_size_per_client

        x_train = x_train_all[train_start:train_end]
        y_train = y_train_all[train_start:train_end]
        x_test = x_test_all[test_start:test_end]
        y_test = y_test_all[test_start:test_end]

        client_dir = os.path.join(output_dir, f"client{i+1}")
        os.makedirs(client_dir, exist_ok=True)

        np.save(os.path.join(client_dir, "x_train.npy"), x_train)
        np.save(os.path.join(client_dir, "y_train.npy"), y_train)
        np.save(os.path.join(client_dir, "x_test.npy"), x_test)
        np.save(os.path.join(client_dir, "y_test.npy"), y_test)

        print(f"\n📦 Client{i+1} - Training Samples: {len(x_train)}")
        print(f"🧪 Client{i+1} - Testing Samples: {len(x_test)}")

# Main execution
if __name__ == "__main__":
    images, labels = load_and_preprocess_images()
    split_and_save(images, labels)


KeyboardInterrupt: 